In [1]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient

from sklearn.decomposition import PCA
from numpy.random import default_rng

# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

query timestamp: 2023-05-25 08:10:01.254304+00:00 (1685002201.254304)


In [2]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [3]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [4]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [5]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Motor neuron table

In [6]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')
pre_to_mn_df.shape
pre_to_mn_df.sum(axis=0).sum()

Found pickle file ./dfs_pre_to_mn/pre_to_mn_df_matched_typed_with_nt_20230531.pkl


208999

In [7]:
mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor_med_venU',
    'tarsus_depressor_noid',
    ]
pre_to_mn_df.head()

# new_idx = utils.sort_segment_fcn_index(pre_to_mn_df.columns)
# pre_to_mn_df = pre_to_mn_df[new_idx]

pre_to_mn_df.head()

side                                                                                                                               L  \
nerve                                                                                                                         Dorsal   
segment                                                                                                                       thorax   
function                                                                                                                       swing   
muscle                                                                           tergopleural_promotor_pleural_promotor_miller_28_30   
rank                                                                                                                               0   
segID                                                                                                             648518346487756866   
cell_class preferred_pool NT  classification_system cell_type segID                                                                    
descending thorax_swing   NaN NaN                   NaN       648518346459693060                                                  0    
                                                              648518346479702678                                                  0    
                                                              648518346478550356                                                  0    
                                                              648518346493883193                                                  0    
                                                              648518346509679817                                                  0    

side                                                                                                 \
nerve                                                                                                 
segment                                                                                               
function                                                                                              
muscle                                                                                                
rank                                                                                                  
segID                                                                            648518346491659326   
cell_class preferred_pool NT  classification_system cell_type segID                                   
descending thorax_swing   NaN NaN                   NaN       648518346459693060                  7   
                                                              648518346479702678                  0   
                                                              648518346478550356                  0   
                                                              648518346493883193                 17   
                                                              648518346509679817                  0   

side                                                                                                 \
nerve                                                                                                 
segment                                                                                               
function                                                                                              
muscle                                                                                                
rank                                                                                                  
segID                                                                            648518346517437482   
cell_class preferred_pool NT  classification_system cell_type segID                                   
descending thorax_swing   NaN NaN                   NaN       64851

In [8]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

# Show example local preMNs to tibia extensor pool

In [9]:
All = slice(None)
femur_df_local = pre_to_mn_df.loc[('local',All,All,All),('L','Leg','femur',All,All,All,All)]
# femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

# mft_df = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['main_tibia_flexor_muscle']]
mft_df = femur_df_local.loc[('local','main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor']]
# mft_df = mft_df.iloc[0:50,:]

# just change the rank label on that one MN
clms = mft_df.columns.to_frame()
clms.loc[('L','Leg','femur','flex','main_tibia_flexor','0')].loc[[False,True],'rank']='1'
# clms.loc[('L','Leg','femur','flex','main_tibia_flexor','0',All),:].loc[[False,True],'rank']
mft_df.columns = pd.MultiIndex.from_frame(clms)
mft_df

ext_df = femur_df_local.loc[('local','tibia_extensor',All,All),mpool_dict['tibia_extensor']]
ext_df


C:\Users\tony\AppData\Local\Temp\ipykernel_7528\462457192.py:6: FutureWarning: The behavior of indexing on a MultiIndex with a nested sequence of labels is deprecated and will change in a future version. `series.loc[label, sequence]` will raise if any members of 'sequence' or not present in the index's second level. To retain the old behavior, use `series.index.isin(sequence, level=1)`
  mft_df = femur_df_local.loc[('local','main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor']]
C:\Users\tony\AppData\Local\Temp\ipykernel_7528\462457192.py:13: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  mft_df.columns = pd.MultiIndex.from_frame(clms)


side                                                                                               L  \
nerve                                                                                            Leg   
segment                                                                                        femur   
function                                                                                      extend   
muscle                                                                                tibia_extensor   
rank                                                                                            seti   
segID                                                                             648518346495797355   
cell_class preferred_pool NT   classification_system cell_type segID                                   
local      tibia_extensor Glu  RD                    24B_      648518346478100437                349   
                          Ach  F                     19B       648518346480658945                191   
                          GABA D                     13A       648518346489565548                188   
                          Ach  A                     22A       648518346498429745                119   
                          Glu  S                     8A_       648518346508809279                226   
                          Ach  A                     22A       648518346486563204                126   
                          Glu  R                     RVD       648518346477007632                186   
                          GABA D                     13A       648518346510015014                139   
                               I                     6B        648518346515252039                 87   
                          Ach  K                     4B        648518346514279723                127   
                          Glu  M                     8A_       648518346481666463                  5   
                          Ach  K                     4B        648518346489993499                117   
                          Glu  RD                    24B_      648518346488876637                 32   
                          Ach  N                     4B-       648518346477005584                 34   
                          GABA D                     13A       648518346482762025                 61   
                                                               648518346490636029                 90   
                          Ach  K                     4B        648518346490040347                 86   
                          GABA D                     13A       648518346498076826                 75   
                          Ach  K                     4B        648518346475602868                 56   
                               N                     4B-       648518346494462440                 34   
                          GABA D                     13A       648518346495674018                 51   
                                                               648518346491990923                 34   
                          Glu  Q                     16B       648518346496797412                 58   
                          Ach  A                     22A       648518346486081988                  6   
                               K                     4B        648518346489040361                 57   
                          Glu  Q                     16B       648518346488892685                 55   
                          Ach  E                     1A        648518346520471350                 13   
                                                               648518346495641506                 11   
                          Glu  Q                     16B       648518346531283738                 67   
                          Ach  T                     23B       648518346480630785                 14   
                          NaN  NaN              

In [10]:
# show examples
def build_sphere_synapses_onto(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        # pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_spheres = ngstbld.SphereMapper(center_column='pre_pt_position', radius_column='radius')
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_spheres,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

def build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

In [11]:
ext_df

side                                                                                               L  \
nerve                                                                                            Leg   
segment                                                                                        femur   
function                                                                                      extend   
muscle                                                                                tibia_extensor   
rank                                                                                            seti   
segID                                                                             648518346495797355   
cell_class preferred_pool NT   classification_system cell_type segID                                   
local      tibia_extensor Glu  RD                    24B_      648518346478100437                349   
                          Ach  F                     19B       648518346480658945                191   
                          GABA D                     13A       648518346489565548                188   
                          Ach  A                     22A       648518346498429745                119   
                          Glu  S                     8A_       648518346508809279                226   
                          Ach  A                     22A       648518346486563204                126   
                          Glu  R                     RVD       648518346477007632                186   
                          GABA D                     13A       648518346510015014                139   
                               I                     6B        648518346515252039                 87   
                          Ach  K                     4B        648518346514279723                127   
                          Glu  M                     8A_       648518346481666463                  5   
                          Ach  K                     4B        648518346489993499                117   
                          Glu  RD                    24B_      648518346488876637                 32   
                          Ach  N                     4B-       648518346477005584                 34   
                          GABA D                     13A       648518346482762025                 61   
                                                               648518346490636029                 90   
                          Ach  K                     4B        648518346490040347                 86   
                          GABA D                     13A       648518346498076826                 75   
                          Ach  K                     4B        648518346475602868                 56   
                               N                     4B-       648518346494462440                 34   
                          GABA D                     13A       648518346495674018                 51   
                                                               648518346491990923                 34   
                          Glu  Q                     16B       648518346496797412                 58   
                          Ach  A                     22A       648518346486081988                  6   
                               K                     4B        648518346489040361                 57   
                          Glu  Q                     16B       648518346488892685                 55   
                          Ach  E                     1A        648518346520471350                 13   
                                                               648518346495641506                 11   
                          Glu  Q                     16B       648518346531283738                 67   
                          Ach  T                     23B       648518346480630785                 14   
                          NaN  NaN              

In [12]:
# Largest 22A neuron
timestamp = connectome_create.get_timestamp()
feti_segid = ext_df.loc[:,(All,All,All,All,All,'feti')].columns.get_level_values('segID').to_list()[0]
seti_segid = ext_df.loc[:,(All,All,All,All,All,'seti')].columns.get_level_values('segID').to_list()[0]

In [13]:

pmn_22A_large = ext_df.loc[('local', 'tibia_extensor',	'Ach','A','22A',All)].index.get_level_values('segID').to_list()[0]
pmn_4B_large = ext_df.loc[('local', 'tibia_extensor',	'Ach','K','4B',All)].index.get_level_values('segID').to_list()[0]
pmn_16B_small = ext_df.loc[('local', 'tibia_extensor',	'Glu','Q','16B',All)].index.get_level_values('segID').to_list()[4]
pmn_8A_small = ext_df.loc[('local', 'tibia_extensor',	'Glu','S','8A_',All)].index.get_level_values('segID').to_list()[1]

In [14]:

render_neurons = [pmn_22A_large,
                  pmn_4B_large,
                  pmn_16B_small,
                  pmn_8A_small,]

for pmn in render_neurons:


    FETi_input_from_extn_ach_pmn = client.materialize.synapse_query(post_ids = feti_segid, pre_ids=pmn, timestamp=timestamp)
    print(FETi_input_from_extn_ach_pmn.shape)

    SETi_input_from_extn_ach_pmn = client.materialize.synapse_query(post_ids = seti_segid, pre_ids=pmn, timestamp=timestamp)
    print(SETi_input_from_extn_ach_pmn.shape)

    syn_df_dict = {'FETi':FETi_input_from_extn_ach_pmn,'SETi':SETi_input_from_extn_ach_pmn}
    clr_dict = {'FETi': '#D38327','SETi':'#FEE39F'}

    # clr_list = ['#0000AA','#2222AA','#4444CC','#6666FF','#AAAAFF']
    # for rnk,id,clr in zip(flexor_rnk,flexor_id,clr_list):
    #     flexor_segid = mn_nt_pool_sorted.loc[(All,'main_tibia_flexor',All,All,All,id),:].index.get_level_values('post_pt_root_id').to_list()
    #     syn_df = client.materialize.synapse_query(post_ids = flexor_segid, pre_ids=flex_glu_pmn_rt_id, timestamp=timestamp)
    #     syn_df.loc[:,'radius'] = int(100)
    #     syn_df_dict[rnk] = syn_df
    #     clr_dict[rnk] = clr

    #build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict)
    # build_sphere_synapses_onto(render_neurons,syn_df_dict,clr_dict)
    # SETi_input_from_extn_ach_pmn

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                    source=client.info.image_source(),
                                    )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = pmn)

    feti_mapper = ngstbld.PointMapper(point_column='pre_pt_position')
    feti_layer = ngstbld.AnnotationLayerConfig(name='FETi_input_from_extn_ach_pmn',
                                    mapping_rules=feti_mapper,
                                    linked_segmentation_layer=seg_layer.name,
                                    color='#D38327')

    feti_sb = ngstbld.StateBuilder(layers=[img_layer, seg_layer, feti_layer], client=client, resolution=[4.3,4.3,45])

    #Second state builder
    seti_mapper = ngstbld.PointMapper(point_column='pre_pt_position')
    seti_layer = ngstbld.AnnotationLayerConfig(name='SETi_input_from_extn_ach_pmn',
                                    mapping_rules=seti_mapper,
                                    linked_segmentation_layer=seg_layer.name,
                                    color='#FEE39F')

    seti_sb = ngstbld.StateBuilder(layers=[seti_layer])


    chained_sb = ngstbld.ChainedStateBuilder([feti_sb, seti_sb])
    state = chained_sb.render_state(data_list=[FETi_input_from_extn_ach_pmn, SETi_input_from_extn_ach_pmn],return_as='dict')
    state_id = client.state.upload_state_json(state)
    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"
    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)
    print(url)

(182, 11)
(119, 11)
https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/4561189050777600
(266, 11)
(127, 11)
https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5677232980754432
(24, 11)
(27, 11)
https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5745795120758784
(17, 11)
(9, 11)
https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5329792272433152


In [15]:
SETi_input_from_extn_ach_pmn

,id,created,superceded_id,valid,score,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position
0,37599453,2022-11-17 14:28:59.843347+00:00,NaN,t,29,72764581088640551,648518346524147717,72764581088638128,648518346495797355,"[22495, 98323, 2131]","[22459, 98317, 2131]"
1,7026014,2022-11-17 14:28:59.843347+00:00,NaN,t,21,72834881113417854,648518346524147717,72764512369280342,648518346495797355,"[22545, 98199, 2128]","[22515, 98189, 2130]"
2,5498241,2022-11-17 14:28:59.843347+00:00,NaN,t,21,72834881113303561,648518346524147717,72834881113296002,648518346495797355,"[23173, 97925, 2081]","[23155, 97907, 2082]"
3,9123369,2022-11-17 14:28:59.843347+00:00,NaN,t,27,72905181205179600,648518346524147717,72905181205201560,648518346495797355,"[25133, 95421, 2252]","[25139, 95391, 2253]"
4,9123373,2022-11-17 14:28:59.843347+00:00,NaN,t,12,72905181205214447,648518346524147717,72905181205201572,648518346495797355,"[25131, 95429, 2256]","[25137, 95401, 2256]"
5,17129522,2022-11-17 14:28:59.843347+00:00,NaN,t,36,72834949698584532,648518346524147717,72834949698550068,648518346495797355,"[24255, 98723, 1864]","[24279, 98743, 1864]"
6,28811846,2022-11-17 14:28:59.843347+00:00,NaN,t,13,72834881113389289,648518346524147717,72834949832869282,648518346495797355,"[22633, 98291, 2121]","[22625, 98309, 2120]"
7,38080058,2022-11-17 14:28:59.843347+00:00,NaN,t,20,72834949765851768,648518346524147717,72834949765847198,648518346495797355,"[23089, 100021, 2032]","[23091, 99993, 2033]"
8,31524265,2022-11-17 14:28:59.843347+00:00,NaN,t,53,72834949698517677,648518346524147717,72834949698495307,648518346495797355,"[23499, 99241, 1840]","[23527, 99233, 1842]"


# Show example local preMNs to tibia flexor pool

In [16]:
All = slice(None)
mft_df = pre_to_mn_df.loc[('local','main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor_wtarsus']]
# femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

# mft_df = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['main_tibia_flexor_muscle']]
# mft_df = femur_df_local.loc[('local','main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor_wtarsus']]
# mft_df = mft_df.iloc[0:50,:]

# just change the rank label on that one MN
clms = mft_df.columns.to_frame()
clms.loc[('L','Leg','femur','flex','main_tibia_flexor','0')].loc[[False,True],'rank']='1'
# clms.loc[('L','Leg','femur','flex','main_tibia_flexor','0',All),:].loc[[False,True],'rank']
mft_df.columns = pd.MultiIndex.from_frame(clms)
mft_df.head(60)



C:\Users\tony\AppData\Local\Temp\ipykernel_7528\4066529991.py:13: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  mft_df.columns = pd.MultiIndex.from_frame(clms)


side                                                                                                      L  \
nerve                                                                                                   Leg   
segment                                                                                               femur   
function                                                                                               flex   
muscle                                                                               auxiliary_tibia_flexor   
rank                                                                                                  Bslow   
segID                                                                                    648518346489731855   
cell_class preferred_pool    NT   classification_system cell_type segID                                       
local      main_tibia_flexor Glu  B                     21A       648518346520630070                     12   
                             Ach  C                     3A        648518346485937022                      4   
                             Glu  B                     21A       648518346496100925                     23   
                                  R                     R1        648518346509612931                     28   
                                  RD                    24B_      648518346498412564                     16   
                             GABA D                     13A       648518346488060446                     16   
                             Glu  B                     21A       648518346465109621                      0   
                             GABA G                     13B       648518346503759265                      4   
                             Ach  E                     1A        648518346466032688                      0   
                                                                  648518346482723625                     35   
                                  C                     3A        648518346498574576                      0   
                             Glu  B                     21A       648518346489729119                      5   
                             Ach  C                     3A        648518346489748575                      0   
                                  A                     22A       648518346486862233                      0   
                                                                  648518346477530844                      0   
                             GABA D                     13A       648518346478750036                      0   
                             Ach  C                     3A        648518346478945488                      0   
                                                                  648518346489658199                      0   
                             Glu  B                     21A       648518346481309327                      0   
                             Ach  A                     22A       648518346489761843                      0   
                                  C                     3A        648518346496348023                     15   
                                  A                     22A       648518346497326104                      0   
                                  C                     3A        648518346502517062                      5   
                                  A                     22A       648518346494311538                      0   
                                                                  648518346496930276                      3   
                                  C                     3A        648518346506663970                     14   
                             NaN  NaN                   NaN       648518346495907403                      0   
                             Glu  P                     14A       648518346499514855                     13   
    

In [17]:
# Largest 22A neuron
timestamp = connectome_create.get_timestamp()
flexor_segid = mft_df.columns.get_level_values('segID').to_list()
flexor_segid

[648518346489731855,
 648518346496059082,
 648518346499176513,
 648518346482926501,
 648518346493836024,
 648518346494269831,
 648518346494196727,
 648518346496932836,
 648518346484809885,
 648518346486906595]

In [18]:

pmn_21A_large = mft_df.loc[('local', 'main_tibia_flexor',	'Glu','B','21A',All)].index.get_level_values('segID').to_list()[0]
pmn_13A_large = mft_df.loc[('local', 'main_tibia_flexor',	'GABA','D','13A',All)].index.get_level_values('segID').to_list()[0]
pmn_3A_medium = mft_df.loc[('local', 'main_tibia_flexor',	'Ach','C','3A',All)].index.get_level_values('segID').to_list()[5]
# pmn_8A_small = ext_df.loc[('local', 'tibia_extensor',	'Glu','S','8A_',All)].index.get_level_values('segID').to_list()[1]
pmn_22A_small = mft_df.loc[('local', 'main_tibia_flexor',	'Ach','A','22A',All)].index.get_level_values('segID').to_list()[-3]


In [19]:

render_neurons = [pmn_21A_large,
                  pmn_13A_large,
                  pmn_3A_medium,
                  pmn_22A_small,]

clr_list = ['#0000AA','#1111AA','#2222AA','#2222CC','#3333CC','#4444CC','#4444DD','#5555FF','#6666FF','#8888CC','#AAAAFF']  

for pmn in render_neurons:
    syn_df_dict = {}
    syn_df_list = [[None]]
    clr_dict = {}
    for flxr,clr in zip(flexor_segid,clr_list[::-1]):
        syn_df = client.materialize.synapse_query(post_ids = flxr, pre_ids=pmn, timestamp=timestamp)
        syn_df_dict[flxr] = syn_df
        syn_df_list = syn_df_list + [syn_df]
        clr_dict[flxr] = clr
        print(syn_df.shape)

    syn_df_dict = {'FETi':FETi_input_from_extn_ach_pmn,'SETi':SETi_input_from_extn_ach_pmn}
    # clr_dict = {'FETi': '#D38327','SETi':'#FEE39F'}
    # 

    #build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict)
    # build_sphere_synapses_onto(render_neurons,syn_df_dict,clr_dict)
    # SETi_input_from_extn_ach_pmn

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                    source=client.info.image_source(),
                                    )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = pmn)
    
    init_sb = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])

    points_sb = []
    for flxr in flexor_segid:
        pt_mapper = ngstbld.PointMapper(point_column='pre_pt_position')
        next_layer = ngstbld.AnnotationLayerConfig(name=str(flxr),
                                    mapping_rules=pt_mapper,
                                    linked_segmentation_layer=seg_layer.name,
                                    color=clr_dict[flxr])
        
        
        points_sb = points_sb + [ngstbld.StateBuilder(layers=[next_layer])]


    chained_sb = ngstbld.ChainedStateBuilder([init_sb] + points_sb)
    state = chained_sb.render_state(data_list=syn_df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)
    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"
    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)
    print(url)

(12, 11)
(20, 11)
(28, 11)
(99, 11)
(276, 11)
(315, 11)
(350, 11)
(498, 11)
(544, 11)
(24, 11)
https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/6300601715523584
(16, 11)
(2, 11)
(18, 11)
(32, 11)
(117, 11)
(78, 11)
(117, 11)
(101, 11)
(144, 11)
(15, 11)
https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5955510891184128
(15, 11)
(5, 11)
(43, 11)
(38, 11)
(15, 11)
(63, 11)
(6, 11)
(13, 11)
(11, 11)
(16, 11)
https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/6507270709968896
(0, 11)
(0, 11)
(1, 11)
(2, 11)
(15, 11)
(7, 11)
(10, 11)
(9, 11)
(7, 11)
(0, 11)
https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5073543584808960


In [20]:
# show examples
def build_sphere_synapses_onto(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        # pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_spheres = ngstbld.SphereMapper(center_column='pre_pt_position', radius_column='radius')
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_spheres,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

def build_ach_synapses_onto_pools(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url